# Project: Using SQL to analyse data about sales and answer business questions
In this project, we will be using SQL to answer business questions. We will work with [Chinook database](https://github.com/lerocha/chinook-database) which is provided as a SQLite database `chinook.db`

`Chinook` database contains many tables. All the tables and the relations between them of the `Chinook` schema is in the below picture
![Chinook_schema](https://s3.amazonaws.com/dq-content/189/chinook-schema.svg)
## 1. Connecting Jupyter Notebook to the database file

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## 2. Overview of the data

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table", "view")

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


As pointed out in the schema summary figure, the database contains 11 tables.

## 3. Making a business decision on which albums to purchase

**Background**:
The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

| Artist Name | Genre |
| ----------- | ------ |
| Regal | Hip-Hop |
| Red Tone | Punk |
| Meteor and the Girls | Pop |
| Slim Jim Bites | Blues |

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

** We have to make a recommendation for the three artists whose albums we should purchase for the store **

We will start looking at which genres sell the most in USA. For this we have to answer two questions. Among the users from USA,
- How many tracks of each genre has been sold ?
- What are the top genres which made more money ?

In [3]:
%%sql

DROP VIEW IF EXISTS tr_gn_info;
DROP VIEW IF EXISTS lk_cst_tr;

/* view for getting track and genre info */ 
CREATE VIEW tr_gn_info AS
    SELECT tr.track_id, tr.genre_id, gn.name, tr.unit_price
    FROM track tr
    INNER JOIN genre gn ON tr.genre_id = gn.genre_id;

/* view for linking customer and track tables */
CREATE VIEW lk_cst_tr AS
    SELECT cst.customer_id, tr.track_id
    FROM customer cst
    INNER JOIN invoice inv ON cst.customer_id = inv.customer_id
    INNER JOIN invoice_line il ON inv.invoice_id = il.invoice_id
    INNER JOIN track tr ON il.track_id = tr.track_id;
    
WITH 
    /* selecting customers from USA */
    cst_usa AS
    (
        SELECT customer_id, country
        FROM customer
        WHERE country = 'USA'
    )

SELECT 
    tr_gn_info.name Genre,
    COUNT(tr_gn_info.track_id) no_tracks_purchased,
    ROUND(SUM(tr_gn_info.unit_price), 2) total_money_sales
FROM cst_usa 
INNER JOIN lk_cst_tr ON cst_usa.customer_id = lk_cst_tr.customer_id
INNER JOIN tr_gn_info ON lk_cst_tr.track_id = tr_gn_info.track_id
GROUP BY tr_gn_info.name
ORDER BY 2 DESC;

Done.
Done.
Done.
Done.
Done.


Genre,no_tracks_purchased,total_money_sales
Rock,561,555.39
Alternative & Punk,130,128.7
Metal,124,122.76
R&B/Soul,53,52.47
Blues,36,35.64
Alternative,35,34.65
Latin,22,21.78
Pop,22,21.78
Hip Hop/Rap,20,19.8
Jazz,14,13.86


### Observations:
The above table suggests total money generated is proportional to the number of tracks sold to the customers from `USA`.

Hence for the options available to our company, following can be tabulated.

| Artist Name | `Genre` | Tracks sold in `Genre` |
| ----------- | ------ | ----------------------------- |
| Regal | Hip-Hop | 20 |
| Red Tone | Punk | 130 |
| Meteor and the Girls | Pop | 22 |
| Slim Jim Bites | Blues | 36 |
### Recommendation:
Based on the available data, it is advisable to go with the following 3 artists
- Red Tone (Punk)
- Slim Jim Bites (Blues)
- Meteor and the Girls (Pop)

## 4. Identifying star employees and their characteristics
Each customer for the `Chinook` store gets assigned to a sales support agent within the company when they first make a purchase. We will analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [4]:
%%sql

/* selecting employees-customer relationship */
WITH
    emp_cst AS
    (
        SELECT
            emp.first_name || ' ' || emp.last_name employee_name,
            emp.employee_id,
            emp.title,
            emp.reports_to,
            emp.birthdate,
            emp.hire_date,
            emp.country,
            cst.customer_id
        FROM employee emp
        LEFT JOIN customer cst ON emp.employee_id = cst.support_rep_id
    )
    
SELECT 
    ROUND(SUM(tr_gn_info.unit_price), 2) total_money_sales,
    emp_cst.*
FROM emp_cst
    LEFT JOIN lk_cst_tr ON emp_cst.customer_id = lk_cst_tr.customer_id
    LEFT JOIN tr_gn_info ON lk_cst_tr.track_id = tr_gn_info.track_id
GROUP BY emp_cst.employee_id
ORDER BY 1 DESC;

Done.


total_money_sales,employee_name,employee_id,title,reports_to,birthdate,hire_date,country,customer_id
1733.51,Jane Peacock,3,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,Canada,59
1585.0,Margaret Park,4,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,Canada,56
1393.92,Steve Johnson,5,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,Canada,57
None,Andrew Adams,1,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,Canada,None
None,Nancy Edwards,2,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,Canada,None
None,Michael Mitchell,6,IT Manager,1,1973-07-01 00:00:00,2016-10-17 00:00:00,Canada,None
None,Robert King,7,IT Staff,6,1970-05-29 00:00:00,2017-01-02 00:00:00,Canada,None
None,Laura Callahan,8,IT Staff,6,1968-01-09 00:00:00,2017-03-04 00:00:00,Canada,None


In [5]:
%%sql
SELECT invoice_date
FROM invoice
ORDER BY 1 DESC
LIMIT 10;

Done.


invoice_date
2020-12-30 00:00:00
2020-12-29 00:00:00
2020-12-27 00:00:00
2020-12-27 00:00:00
2020-12-21 00:00:00
2020-12-20 00:00:00
2020-12-20 00:00:00
2020-12-19 00:00:00
2020-12-18 00:00:00
2020-12-17 00:00:00


### Observations:
- The data shows only 8 employees in total for the company.
- Of those 8, only 3 are Sales Agents and they all 3 report to Sales Manager.
- The last sale is in December 2020 according to the data and all the three Sales agents have been hired in 2017, albeit at various times.  
- So, even if we take average amount of sales carried out per month, both the ladies score higher than `Steve`, although not by much.
- Ms. `Jane Peacock` is clearly the star employee of the sales department.

| Employee | Months in service | Total amt sales | Avg amt sales/month |
| ----- | ---- | ----| --- |
| Jane Peacock | 44 | 1733 | 39.38 |
| Margaret Park | 43 | 1585 | 36.86 |
| Steve Johnson | 38 | 1393 | 36.65 |

## 5. Analysing sales by country
Our next task is to analyze the sales data for customers from each different country. We will use the `country` value from the customers table, and ignore the `country` from the billing address in the invoice table.

For each country, in the database, we will calculate
- total number of customers
- total value of sales
- average value of sales per customer
- average order value

In [6]:
%%sql

/* linking customer and invoice tables */
WITH cst_inv AS
(
    SELECT 
        CASE
            WHEN
                (
                    SELECT COUNT(*)
                    FROM customer
                    WHERE country = cst.country
                ) = 1 THEN 'Others'
            ELSE cst.country
        END AS country_col,      
        cst.customer_id,
        SUM(inv.total) total,
        COUNT(inv.invoice_id) no_orders
    FROM customer cst
    INNER JOIN invoice inv ON cst.customer_id = inv.customer_id
    GROUP BY cst.customer_id
)

SELECT 
    country_col,
    COUNT(customer_id) total_no_customers,
    ROUND(SUM(CAST(total AS FLOAT)), 2) total_sales_USD,
    ROUND(SUM(CAST(total AS FLOAT)) / COUNT(customer_id), 2) avg_sales_customer,
    ROUND(SUM(CAST(total AS FLOAT)) / SUM(no_orders), 2) avg_order_val/* average order value = total sales / no of orders */
FROM
    (
        SELECT *,
            CASE
                WHEN country_col = 'Others' THEN 1
                ELSE 0
            END AS sort
        FROM cst_inv
    )
GROUP BY country_col
ORDER BY sort, 2 DESC;
    

Done.


country_col,total_no_customers,total_sales_USD,avg_sales_customer,avg_order_val
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
United Kingdom,3,245.52,81.84,8.77
Czech Republic,2,273.24,136.62,9.11
India,2,183.15,91.57,8.72
Portugal,2,185.13,92.56,6.38
Others,15,1094.94,73.0,7.45


Everything looks good, except that there are many countries with only a single customer. It would be be good to
- Club all the countries with only a single customer into `Others` category
- This `Others` category should be listed at the last of the table

### Observations:
- By far, USA generates the bulk of sales followed by Canada and Brazil.
- It is interesting to know that countries of UK, Czech Republic and India have higher average sales / customer and average order values. However, it has to be noted that there are very few samples for these countries to formulate any kind of hypotheses.
- If we have some money, small surveys could be carried out in these countries to generate more data to make an informed decision

## 6. Business decision to change purchase strategy : Albums vs Individual Tracks
The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:
- purchase a whole album
- purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

**Possible business strategy**:

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We are asked to give our recommendation.

**Assumption**:

It is very common when we are performing an analysis to have 'edge cases' which prevent us from getting a 100% accurate answer to the question. In this instance, we have two edge cases to consider:
- Albums that have only one or two tracks are likely to be purchased by customers as part of a collection of individual tracks.
- Customers may decide to manually select every track from an album, and then add a few individual tracks from other albums to their purchase.

In the first case, since our analysis is concerned with maximizing revenue we can safely ignore albums consisting of only a few tracks. The company has previously done analysis to confirm that the second case does not happen often, so we can ignore this case also.


In [7]:
%%sql
WITH invoice_track_info AS
(
SELECT invoice_id, MIN(track_id) first_track_id
FROM invoice_line
GROUP BY 1
),
inv_album_or_not_table AS
(
    SELECT
    in_tr.invoice_id,
        CASE 
            WHEN
                (
                    SELECT 
                        track_id 
                    FROM (SELECT tr.track_id, tr.album_id
                            FROM track tr
                            WHERE tr.album_id = (
                                                    SELECT album_id
                                                    FROM track
                                                    WHERE track_id = in_tr.first_track_id
                                                )
                         )
                    EXCEPT
                    SELECT track_id 
                    FROM (
                            SELECT il.invoice_id, il.track_id
                            FROM invoice_line il
                            WHERE il.invoice_id = in_tr.invoice_id
                          )
                ) IS NULL
                AND
                (
                    SELECT track_id
                    FROM (
                            SELECT il.invoice_id, il.track_id
                            FROM invoice_line il
                            WHERE il.invoice_id = in_tr.invoice_id
                        )
                    EXCEPT
                    SELECT 
                        track_id
                    FROM (SELECT tr.track_id, tr.album_id
                            FROM track tr
                            WHERE tr.album_id = (
                                                    SELECT album_id
                                                    FROM track
                                                    WHERE track_id = in_tr.first_track_id
                                                )
                         )
                ) IS NULL
                THEN "Yes"
            ELSE "No"
        END AS invoice_album_or_not

    FROM invoice_track_info AS in_tr
)

SELECT 
    invoice_album_or_not,
    count(invoice_album_or_not) tot_number,
    ROUND(CAST(count(invoice_album_or_not) AS FLOAT)*100/(SELECT COUNT(*) FROM inv_album_or_not_table),2) AS percentage
FROM inv_album_or_not_table
GROUP BY invoice_album_or_not;

Done.


invoice_album_or_not,tot_number,percentage
No,500,81.43
Yes,114,18.57


### Recommendation:
- It can be observed that 18.57% of total sales are album purchases.
- If the management decides to purchase only the hit tracks, instead of entire albums, bulk of this 18.57% sales will be effected.

**Hence it is recommended not to go ahead with that decision**

## 7. Miscellaneous
Few more questions asked at the end are
- Which artist is used in the most playlists?
- How many tracks have been purchased vs not purchased?
- Is the range of tracks in the store reflective of their sales popularity?
- Do protected vs non-protected media types have an effect on popularity?

In [19]:
%%sql
WITH playlist_info AS
(
    SELECT ar.artist_id artist_id, ar.name name
    FROM playlist_track pl_tr
    INNER JOIN track tr ON pl_tr.track_id = tr.track_id
    INNER JOIN album alb ON tr.album_id = alb.album_id
    INNER JOIN artist ar ON alb.artist_id = ar.artist_id
)

SELECT COUNT(artist_id) no_times, name
FROM playlist_info
GROUP BY artist_id
ORDER BY 1 DESC
LIMIT 10;


Done.


no_times,name
516,Iron Maiden
333,U2
296,Metallica
252,Led Zeppelin
226,Deep Purple
184,Lost
177,Pearl Jam
145,Eric Clapton
145,Faith No More
143,Lenny Kravitz


### a) Which artist is used in the most playlists?
Answer: It can be seen that Iron Maiden is in most of the playlists

### b) How many tracks have been purchased vs not purchased?

In [30]:
%%sql
WITH invoice_info AS
(
SELECT track_id AS track_invoice FROM invoice_line
)

SELECT
    COUNT(track_id) no_times,
    CASE
        WHEN track_id in invoice_info
        THEN "Yes"
        ELSE "No"
    END AS purchased_or_not,
    ROUND(CAST(COUNT(track_id) * 100 AS FLOAT)/(SELECT COUNT(track_id) FROM track), 2) percentage
FROM track
GROUP BY purchased_or_not;

Done.


no_times,purchased_or_not,percentage
1697,No,48.44
1806,Yes,51.56


**Answer**: Almost half of the tracks are never purchased. Now this becomes interesting. How many of the tracks were purchased more than 10 times, 100 times, 1000 times etc.
### c) Is the range of tracks in the store reflective of their sales popularity?

In [42]:
%%sql
WITH track_purchases_info AS
(
SELECT
    COUNT(inv_li.track_id) AS track_purchased_times,
    inv_li.track_id,
    tr.name,
    tr.composer
FROM invoice_line inv_li
    INNER JOIN track tr ON inv_li.track_id = tr.track_id
GROUP BY inv_li.track_id
)

SELECT * FROM track_purchases_info ORDER BY 1 DESC LIMIT 50;

Done.


track_purchased_times,track_id,name,composer
31,3336,War Pigs,None
14,1489,Are You Experienced?,Jimi Hendrix
14,1495,Highway Chile,Jimi Hendrix
13,6,Put The Finger On You,"Angus Young, Malcolm Young, Brian Johnson"
13,1487,Third Stone From The Sun,Jimi Hendrix
13,1490,Hey Joe,Billy Roberts
12,1483,Love Or Confusion,Jimi Hendrix
12,2558,Radio/Video,"Dolmayan, John/Malakian, Daron/Odadjian, Shavo"
11,1129,Dead And Broken,Sully Erna
11,1479,Foxy Lady,Jimi Hendrix


**Observations**:
- Looks like HeavyMetal/Rock tracks are top sellers.
- Top two songs are on War theme.
- Overall there are only 59 users in the database, yet `War Pigs` was purchased by more than half of them. Interesting, may be the purchasers are Vietnam war veterans from USA ??? Lets, find out.

In [54]:
%%sql
/* Users who purchased War Pigs song */
SELECT
    /*cst.first_name || ' ' || cst.last_name AS user_name,*/
    COUNT(cst.customer_id),
    cst.country
FROM customer cst
    INNER JOIN invoice inv ON cst.customer_id = inv.customer_id
    INNER JOIN invoice_line inv_li ON inv.invoice_id = inv_li.invoice_id
    INNER JOIN track tr ON inv_li.track_id = tr.track_id
WHERE tr.track_id = 3336
GROUP BY country
ORDER BY 1 DESC;


Done.


COUNT(cst.customer_id),country
6,USA
2,Czech Republic
2,France
2,Germany
2,India
2,Portugal
2,United Kingdom
1,Argentina
1,Belgium
1,Brazil


Lol, our assumption is wrong. Only 6 purchases of 31 are from USA.
### d) Do protected vs non-protected media types have an effect on popularity?

In [65]:
%%sql
WITH track_purchases_info AS
(
SELECT
    COUNT(inv_li.track_id) no_purchases,
    inv_li.track_id,
    inv_li.invoice_line_id inv_line_id,
    tr.media_type_id media_type_id,
    med_typ.name media_type_desc
FROM invoice_line inv_li
    INNER JOIN track tr ON inv_li.track_id = tr.track_id
    INNER JOIN media_type med_typ ON tr.media_type_id = med_typ.media_type_id
GROUP BY inv_li.track_id
)

SELECT SUM(no_purchases) total_purchases, media_type_id, media_type_desc 
FROM track_purchases_info 
GROUP BY media_type_desc 
ORDER BY 1 DESC;

Done.


total_purchases,media_type_id,media_type_desc
4259,1,MPEG audio file
439,2,Protected AAC audio file
35,4,Purchased AAC audio file
21,5,AAC audio file
3,3,Protected MPEG-4 video file


**Answer is No. The most popular media type by far is `MPEG audio file` and not the latest `MPEG-4 Video`.**